In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [19]:
indicadores = pd.read_excel("../wrangled_data/indicadores.xlsx", index_col = "Data")
btc = pd.read_excel("../wrangled_data/serie_bitcoin.xlsx", index_col = "Data")

In [11]:
inds_macro_sem = pd.read_excel("../raw_data/Dados Macro Semanal.xlsx", index_col = "Data")
inds_macro_sem.head()

,SP500,DXY,GLD,10YIELD
Data,,,,
2011-02-06,0.013945,0.005382,0.005013,0.000000
2011-02-13,0.008487,-0.010196,0.023352,-0.015393
2011-02-20,-0.015331,-0.004893,0.014548,-0.046343
2011-02-27,0.000962,-0.011387,0.014340,0.022248
2011-03-06,-0.012769,0.004974,-0.008109,-0.025773


# Convertendo BTC

In [40]:
inds_tec = pd.DataFrame()
inds_tec["Retorno"] = btc["Fechamento"].pct_change()

# Desvios-Padrão Móveis
inds_tec["STD5"] = inds_tec["Retorno"].rolling(5).std()
inds_tec["STD10"] = inds_tec["Retorno"].rolling(10).std()
inds_tec["STD15"] = inds_tec["Retorno"].rolling(15).std()

inds_tec["MM10"] = btc["Fechamento"].rolling(10).mean()

# Proporções de Corpo dos Candles
inds_tec["PROP"] = (btc["Fechamento"]-btc["Abertura"])/(btc["Máxima"]-btc["Mínima"])

# Direções dos Dias Anteriores
inds_tec["D0"] = np.where(btc["Fechamento"] > btc["Abertura"], 1, 0)
inds_tec["D1"] = inds_tec["D0"].shift(1)
inds_tec["D2"] = inds_tec["D0"].shift(2)
inds_tec["D3"] = inds_tec["D0"].shift(3)

# Z-Score Móvel
tmp = pd.DataFrame()
tmp["MM30"] = btc["Fechamento"].rolling(30).mean()
tmp["MM200"] = btc["Fechamento"].rolling(200).mean()
tmp["STD200"] = btc["Fechamento"].rolling(200).std()
inds_tec["Z30"] = (tmp["MM30"]-tmp["MM200"])/tmp["STD200"]

# RSL
inds_tec["RSL"] = (btc["Fechamento"]/inds_tec["MM10"])-1

# Tendência
tmp = pd.DataFrame()
tmp["MM10"] = inds_tec["MM10"]
tmp["MM100"] = btc["Fechamento"].rolling(100).mean()
tmp["MM200"] = btc["Fechamento"].rolling(200).mean()
tmp.dropna(inplace = True)
tmp["TEND_ALTA"] = np.where((tmp["MM10"] > tmp["MM100"]) & (tmp["MM100"] > tmp["MM200"]), 1, 0)
tmp["TEND_BAIXA"] = np.where((tmp["MM10"] < tmp["MM100"]) & (tmp["MM100"] < tmp["MM200"]), 1, 0)
tmp.drop(["MM10", "MM100", "MM200"], axis = 1, inplace = True)
inds_tec = inds_tec.join(tmp)

inds_tec.drop(["Retorno", "MM10"], axis = 1, inplace = True)

# Convertendo Indicadores

In [39]:
df

,AN_INFLATION,END_ATIVOS,HASH_RATE30,FEE_MEAN,MMULT,MRGO140,MRGO28,MVRV,MVRV_Z,THERMOCAP,...,STD15,PROP,D0,D1,D2,D3,Z30,RSL,TEND_ALTA,TEND_BAIXA
Data,,,,,,,,,,,,,,,,,,,,,
2009-01-03,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-04,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-05,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-06,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-07,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-13,1.658383,834925,2.021100e+08,70.004088,NaN,NaN,NaN,NaN,NaN,NaN,...,0.017532,0.062894,1.0,1.0,1.0,1.0,-1.135645,0.036399,0.0,1.0
2022-08-14,1.586729,811197,2.014597e+08,65.429275,NaN,NaN,NaN,NaN,NaN,NaN,...,0.017465,-0.134283,0.0,1.0,1.0,1.0,-1.113305,0.024601,0.0,1.0
2022-08-15,1.729810,922140,2.010798e+08,85.083070,NaN,NaN,NaN,NaN,NaN,NaN,...,0.017152,-0.134388,0.0,0.0,1.0,1.0,-1.092869,0.013245,0.0,1.0


In [57]:
week_indicadores = pd.DataFrame()

week_indicadores[["HASH_RATE30", "AN_INFLATION", "FEE_MEAN", "DAYS_HALVING"]] = indicadores[["HASH_RATE30", "AN_INFLATION", "FEE_MEAN", "DAYS_HALVING"]].loc[inds_macro_sem.index].pct_change()
week_indicadores[["TRENDS", "NUPL", "END_ATIVOS", "CDD"]] = indicadores[["TRENDS", "NUPL", "END_ATIVOS", "CDD"]].rolling(7).sum().loc[inds_macro_sem.index]

tmp = pd.DataFrame()
tmp["MM30"] = week_indicadores["END_ATIVOS"].rolling(4).mean()
tmp["MM200"] = week_indicadores["END_ATIVOS"].rolling(29).mean()
tmp["STD200"] = week_indicadores["END_ATIVOS"].rolling(29).std()
week_indicadores["END_ATIVOS"] = (tmp["MM30"] - tmp["MM200"])/tmp["STD200"]

tmp = pd.DataFrame()
tmp["MM30"] = week_indicadores["CDD"].rolling(4).mean()
tmp["MM200"] = week_indicadores["CDD"].rolling(29).mean()
tmp["STD200"] = week_indicadores["CDD"].rolling(29).std()
week_indicadores["CDD"] = (tmp["MM30"] - tmp["MM200"])/tmp["STD200"]

week_indicadores[["MMULT", "MRGO140", "MRGO28", "MVRV", "MVRV_Z", "THERMOCAP", "SOPR"]] = indicadores[["MMULT", "MRGO140", "MRGO28", "MVRV", "MVRV_Z", "THERMOCAP", "SOPR"]].rolling(7).mean().loc[inds_macro_sem.index]
week_indicadores = week_indicadores.join(inds_macro_sem)
week_indicadores = week_indicadores.join(inds_tec)
week_indicadores.drop(pd.to_datetime("2011-09-11"), axis = 0, inplace = True)
week_indicadores

,HASH_RATE30,AN_INFLATION,FEE_MEAN,DAYS_HALVING,TRENDS,NUPL,END_ATIVOS,CDD,MMULT,MRGO140,...,STD15,PROP,D0,D1,D2,D3,Z30,RSL,TEND_ALTA,TEND_BAIXA
Data,,,,,,,,,,,,,,,,,,,,,
2011-02-06,NaN,NaN,NaN,NaN,5.0,5.375773,NaN,NaN,NaN,3.598829,...,0.093246,-0.291073,0,1.0,1.0,0.0,1.714571,0.348527,1.0,0.0
2011-02-13,0.143670,0.104870,-0.748949,-0.010590,27.0,5.577485,NaN,NaN,4.684704,2.934240,...,0.102373,-0.018364,0,1.0,1.0,0.0,1.833351,0.084408,1.0,0.0
2011-02-20,0.195403,-0.013301,0.297338,-0.010703,25.0,5.335417,NaN,NaN,3.963747,1.855970,...,0.079547,-0.831905,0,1.0,0.0,0.0,1.995870,-0.157727,1.0,0.0
2011-02-27,0.284646,0.214558,0.008852,-0.010819,23.0,5.114570,NaN,NaN,3.253444,1.228369,...,0.066491,-1.000000,0,1.0,0.0,1.0,2.124256,-0.017164,1.0,0.0
2011-03-06,0.307610,0.108903,-0.361255,-0.010938,20.0,4.774245,NaN,NaN,2.946913,0.874086,...,0.057450,-0.232397,0,1.0,0.0,1.0,2.111906,-0.028515,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-01,0.061470,0.006162,0.063405,-0.006944,161.0,3.553272,-1.547684,0.363068,0.896704,-3.766378,...,0.035509,-0.500544,0,0.0,1.0,0.0,-0.991223,0.046033,0.0,1.0
2021-08-08,0.043215,0.025601,0.013626,-0.006993,133.0,3.612777,-1.408056,0.408439,0.921605,-3.531997,...,0.033467,-0.371278,0,1.0,1.0,1.0,-0.848911,0.059644,0.0,0.0
2021-08-15,0.034100,0.151476,0.002937,-0.007042,152.0,3.951449,-1.200655,-0.000335,1.023157,-2.083597,...,0.036757,-0.022149,0,0.0,1.0,0.0,-0.605543,0.033940,0.0,0.0


In [58]:
week_indicadores.describe().T

,count,mean,std,min,25%,50%,75%,max
HASH_RATE30,552.0,3.956429e-02,6.471347e-02,-1.133699e-01,7.205943e-03,2.380995e-02,0.051780,0.460505
AN_INFLATION,552.0,7.574992e-03,1.650850e-01,-7.000692e-01,-9.925054e-02,-6.940015e-03,0.105256,0.998450
FEE_MEAN,552.0,4.809920e-01,5.257350e+00,-9.999999e-01,-1.417755e-01,5.310400e-03,0.161051,99.888985
DAYS_HALVING,552.0,3.795821e+00,6.480247e+01,-8.750000e-01,-2.124446e-02,-1.071978e-02,-0.006492,1441.000000
TRENDS,553.0,2.168752e+02,1.038924e+02,5.000000e+00,1.580000e+02,2.130000e+02,280.000000,549.000000
NUPL,553.0,2.308418e+00,2.226601e+00,-8.877182e+00,1.477569e+00,2.838535e+00,3.823385,5.827601
END_ATIVOS,525.0,6.739051e-01,9.333439e-01,-1.964584e+00,3.756520e-03,8.733300e-01,1.398915,2.195873
CDD,525.0,8.012186e-02,6.851217e-01,-1.327271e+00,-4.015710e-01,-4.665193e-02,0.485685,1.886607
MMULT,552.0,1.427408e+00,9.828971e-01,2.474340e-01,8.958712e-01,1.170152e+00,1.615663,9.774901
MRGO140,551.0,9.093654e-01,2.709280e+00,-6.669555e+00,-5.317461e-01,3.384176e-01,1.893660,12.858209


# Exportação

In [59]:
week_indicadores.to_excel("../wrangled_data/week_indicadores.xlsx", index_label = "Data")